<div style="text-align:center; font-family: Cambria, 'Times New Roman', serif; color: black;">


  <h1 style="font-family: Arial, sans-serif; font-size: 30pt; margin: 40px 0 10px 0;">
    Automatisierte Erkennung von Pflanzenkrankheiten
  </h1>
  

</div>



Die frühzeitige Erkennung von Pflanzenkrankheiten ist entscheidend für die Sicherung landwirtschaftlicher Erträge. Maschinelles Lernen (ML) bietet moderne Methoden, um den Gesundheitszustand von Pflanzen automatisiert zu klassifizieren und dadurch wirtschaftliche Schäden zu minimieren.

In dieser Arbeit werden klassische ML-Modelle sowie künstliche neuronale Netze (ANN, CNN) auf Bilddaten aus dem PlantVillage-Datensatz angewendet und verglichen. Ziel ist es, die Effektivität verschiedener Modellierungsansätze zur Erkennung gesunder und kranker Blätter – sowohl einzeln pro Pflanzentyp als auch in einem kombinierten Klassifikator – zu evaluieren. Neben der praktischen Umsetzung wird ein besonderer Fokus auf den mathematischen Hintergrund neuronaler Netzwerke gelegt.


Die Landwirtschaft steht vor der Herausforderung, Pflanzenkrankheiten frühzeitig zu erkennen, um Ernteausfälle zu verhindern und den Einsatz von Pflanzenschutzmitteln effizient zu gestalten. Besonders Kleinbauern sind von solchen Verlusten wirtschaftlich stark betroffen. Eine automatisierte Erkennung von Krankheitsbildern mithilfe von Bildanalyse kann in diesem Zusammenhang eine entscheidende Rolle spielen.

Maschinelles Lernen (ML) hat sich als leistungsfähige Methode zur Klassifikation und Mustererkennung etabliert – auch in der Pflanzenpathologie. Insbesondere durch den Einsatz von Deep-Learning-Modellen können Bilder von Pflanzenblättern analysiert und der Gesundheitszustand der Pflanze präzise bestimmt werden. Dies ermöglicht eine frühzeitige Diagnose und gezielte Gegenmaßnahmen zur Ertragssicherung.

## Verwendete Bibliotheken

In [ ]:
# System- und Dateiverwaltung
import os                 # Interaktion mit dem Betriebssystem (Pfade, Ordner)
import shutil             # Datei- und Verzeichnisoperationen (Kopieren, Verschieben)
from PIL import Image     # Bildbearbeitung
import cv2                # Computer Vision (OpenCV)
import pywt               # Wavelet-Transformationen

# Basis-Bibliotheken & Visualisierung
import numpy as np        # Numerische Berechnungen (Arrays, Matrizen)
import random
import pandas as pd       # Datenanalyse und -manipulation (DataFrames)
import matplotlib.pyplot as plt  # Datenvisualisierung (Diagramme)
import matplotlib.image as mpimg # Bilder laden/anzeigen

# Preprocessing, Split & Evaluation (Scikit-learn)
from sklearn.model_selection import train_test_split, GridSearchCV, KFold # Datensatz-Splitting, Hyperparameter-Optimierung, Kreuzvalidierung
from sklearn.preprocessing import StandardScaler, LabelEncoder             # Daten skalieren, Labels kodieren
from sklearn.metrics import (                                             # Metriken zur Modellbewertung 
    confusion_matrix,      
    ConfusionMatrixDisplay,
    accuracy_score,
    classification_report
)

# Klassische ML-Modelle & Pipeline (Scikit-learn)
from sklearn.linear_model import LogisticRegression  
from sklearn.svm import SVC                          
from sklearn.tree import DecisionTreeClassifier      
from sklearn.ensemble import RandomForestClassifier   
from sklearn.pipeline import make_pipeline             # Erstellung von Verarbeitungspipelines
import joblib                                          # Speichern/Laden von Python-Objekten (Modellen)

# Deep Learning (Keras mit TensorFlow)
import tensorflow as tf                                # Haupt-Deep-Learning-Framework
from keras import models, layers                       # Modelldefinition (Schichten, Architektur)
from keras.optimizers import AdamW                     # Optimierungsalgorithmus
from keras.callbacks import EarlyStopping, ReduceLROnPlateau # Callback für Training (z.B. Frühzeitiges Stoppen, Lernraten-Anpassung)
from keras.models import load_model                    # Laden von gespeicherten Keras-Modellen
from tensorflow.keras import backend as K

## Künstliches Neuronales Netz – Eigenständige Implementierung

In [ ]:
def initialize_params():
    """
    Initialisiert Gewichte und Biases mit Zufallswerten und Nullen.
    """

    # W1: Gewichtsmatrix von der Eingabe- (2 Features) zur ersten versteckten Schicht (2 Neuronen)
    w1 = np.random.randn(2, 2) * np.sqrt(2 / 2) 

    # b1: Bias-Vektor für die erste versteckte Schicht (2 Neuronen)
    b1 = np.zeros((1, 2))  

    # W2: Gewichtsmatrix von der ersten versteckten Schicht (2 Neuronen) zur Ausgabe-Schicht (1 Neuron)
    w2 = np.random.randn(2, 1) * np.sqrt(2 / 2)

    # b2: Bias-Vektor für die Ausgabe-Schicht (1 Neuron)
    b2 = np.zeros((1, 1))                     
    return w1, w2, b1, b2

def relu(Z):
    """ReLU Aktivierungsfunktion"""
    return np.maximum(0, Z)

def relu_derivative(Z):
    """Ableitung der ReLU Funktion"""
    return (Z > 0).astype(float)

def sigmoid(Z):
    """Sigmoid Aktivierungsfunktion"""
    return 1 / (1 + np.exp(-Z))

def forward(X, w1, w2, b1, b2):
    """
    Forward-Pass: Berechnet Zwischenergebnisse und Vorhersagen.
    """

    # Erste Schicht
    z1 = np.dot(X, w1) + b1
    a1 = relu(z1)   

    # Zweite (Ausgabe-) Schicht
    z2 = np.dot(a1, w2) + b2
    a2 = sigmoid(z2)

    return z1, a1, z2, a2

def train(X, Y, epochs=10000, learning_rate=0.1):  # Das Netzwerk wird über 'epochs' Iterationen trainiert
    """
    Trainiert das Netzwerk mit Gradient Descent und Backpropagation.
    """
    w1, w2, b1, b2 = initialize_params()
    
    for _ in range(epochs):

        # Forward-Pass
        z1, a1, z2, a2 = forward(X, w1, w2, b1, b2)
        
         # Backpropagation
        # Fehler am Ausgang (Output-Schicht). Die Sigmoid-Funktion steht am Ausgang.
        delta2 = a2 - Y # Dies ist der Gradient des Binary Cross-Entropy-Verlustes bzgl. Z2 für Sigmoid-Output
        
        
        # Gradienten für w2, b2
        dW2 = np.dot(a1.T, delta2)   # Matrixmultiplikation: Transponierte A1 und delta2
        db2 = np.mean(delta2, axis=0, keepdims=True) # Mittelwert von delta2 über die Samples
        
      # Fehler zurück zu Schicht 1 propagieren
        delta1 = np.dot(delta2, w2.T) * relu_derivative(z1) # Elementweises Produkt mit Ableitung der ReLU-Aktivierung
        
        # Gradienten für w1, b1
        dW1 = np.dot(X.T, delta1)
        db1 = np.mean(delta1, axis=0, keepdims=True)
        
        # Parameter-Update (Gradientenabstieg)
        w2 -= learning_rate * dW2
        b2 -= learning_rate * db2
        w1 -= learning_rate * dW1
        b1 -= learning_rate * db1
    
    # Nach 'epochs' Trainingsdurchläufen werden die aktualisierten Gewichte/Biases zurückgegeben,
    # bereit für die Vorhersage der Daten.
    
    return w1, b1, w2, b2

def predict(X, w1, w2, b1, b2):
    """
    Nutzt das trainierte Netzwerk, um Vorhersagen zu treffen.
    """
    
    # Führt den Forward-Pass aus, um die finalen Aktivierungen (a2) zu erhalten
    _, _, _, a2 = forward(X, w1, w2, b1, b2) 
    # Konvertiert die Wahrscheinlichkeiten in binäre Vorhersagen (0 oder 1)
    return (a2 > 0.5).astype(int)

# Beispiel-Daten XOR
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
Y = np.array([[0], [1], [1], [0]])

# --- Training des Netzwerks ---
print("Starte Training des XOR-Netzwerks...")
w1, b1, w2, b2 = train(X, Y)
print("Training abgeschlossen.")

# --- Vorhersage mit dem trainierten Netzwerk ---
predictions = predict(X, w1, w2, b1, b2)
print("\n--- Vorhersagen für XOR-Eingaben ---")
print(predictions)


##  Datensatz & Preprocessing
### Datenauswahl und -aufbereitung

In [ ]:
# Pfad zum ursprünglichen PlantVillage-Datensatz
global_source_dir = 'data/PlantVillage'

# Zielverzeichnis für die gefilterten und strukturierten Bilddaten
global_target_dir = 'data_selected'

# Auswahl: 3 Pflanzenarten (Tomate, Apfel, Mais) mit jeweils 3 Klassen (gesund + 2 Krankheiten)
selected_categories = [
    'Tomato___healthy',
    'Tomato___Bacterial_spot',
    'Tomato___Late_blight',
    'Apple___healthy',
    'Apple___Apple_scab',
    'Apple___Black_rot',
    'Corn___healthy',
    'Corn___Common_rust',
    'Corn___Northern_Leaf_Blight'
]

# Bildgröße zur Weiterverarbeitung – ursprünglich 256x256, hier auf 64x64 reduziert,
# um die Trainings- und Rechenzeit deutlich zu verringern
global_image_size = (64, 64)

# Detailtiefe der Wavelet-Transformation (für spätere Feature-Extraktion)
global_w2d_level = 5

In [ ]:
def select_categories_grouped(source_dir, target_dir, categories):
    
    # Prüfen, ob der Quellordner existiert
    if not os.path.exists(source_dir):
        print("Datenordner existiert nicht!")
        return 
    
    # Zielordner neu anlegen (falls vorhanden, wird er gelöscht)
    if os.path.exists(target_dir):
        shutil.rmtree(target_dir)
    os.makedirs(target_dir)
    
    total_images = 0

    # Schleife über alle ausgewählten Kategorien
    for category in categories:

        # Pfad zur Quellkategorie, z. B. data/PlantVillage/Tomato___healthy
        src_path = os.path.join(source_dir, category)
        
        # Pflanzenname extrahieren, z. B. 'Tomato' aus 'Tomato___healthy'
        plant_name = category.split('___')[0].lower()
        
        # Zielpfad, z. B. data_selected/tomato/raw_images/Tomato___healthy
        dst_dir = os.path.join(target_dir, plant_name, 'raw_images', category)
        
        if os.path.exists(src_path):
            os.makedirs(dst_dir, exist_ok=True)
            
            image_count = 0  # Zähler für Bilder in dieser Kategorie
            
            # Alle Bilddateien einzeln kopieren, um die Zielstruktur sauber aufzubauen
            for filename in os.listdir(src_path):
                # z. B. data/PlantVillage/Tomato___healthy/image (1).JPG
                src_file = os.path.join(src_path, filename)
                # z. B. data_selected/tomato/raw_images/Tomato___healthy/image (1).JPG
                dst_file = os.path.join(dst_dir, filename)
                if os.path.isfile(src_file):
                    shutil.copy2(src_file, dst_file)
                    image_count += 1
            
            total_images += image_count
            print(f"Kategorie '{category}': {image_count} Bilder nach '{dst_dir}' kopiert.")
        else:
            print(f"Achtung: Kategorie '{category}' nicht gefunden!")
    
    print(f"\nInsgesamt kopierte Bilder: {total_images}")


# Funktionsaufruf mit den festgelegten Parametern
select_categories_grouped(
    source_dir=global_source_dir,
    target_dir=global_target_dir,
    categories=selected_categories
)

Zur explorativen Datenanalyse wird aus jeder ausgewählten Klasse ein zufälliges Bild visualisiert. Dies dient der Veranschaulichung der visuellen Unterschiede zwischen den Krankheitskategorien.

In [ ]:
# Sortieren nach Klassenname
all_classes = [
    'data_selected/apple/raw_images/Apple___Apple_scab',
    'data_selected/apple/raw_images/Apple___Black_rot',
    'data_selected/apple/raw_images/Apple___healthy',
    'data_selected/corn/raw_images/Corn___Northern_Leaf_Blight',
    'data_selected/corn/raw_images/Corn___Common_rust',
    'data_selected/corn/raw_images/Corn___healthy',
    'data_selected/tomato/raw_images/Tomato___Late_blight',
    'data_selected/tomato/raw_images/Tomato___Bacterial_spot',
    'data_selected/tomato/raw_images/Tomato___healthy',
]
all_classes = sorted(all_classes)


plt.figure(figsize=(8, 8))
for i, class_path in enumerate(all_classes):
    class_name = os.path.basename(class_path)

    # Zufallszahl zwischen 1 und 100
    rand_index = random.randint(1, 100)
    
    # Dateiname zusammensetzen
    img_name = f"image ({rand_index}).JPG"
    img_path = os.path.join(class_path, img_name)
    print(img_path)

    try:
        img = mpimg.imread(img_path)
        plt.subplot(3, 3, i + 1)
        plt.imshow(img)
        plt.title(class_name.replace('___', '\n'), fontsize=10)
        plt.axis('off')
    except FileNotFoundError:
        print(f"Bild {img_name} in {class_name} nicht gefunden.")

plt.tight_layout()
plt.show()

***Feature-Reduktion durch Wavelet-Transformation***

Die Wavelet-Transformation (z. B. mit Haar-Wavelets) extrahiert strukturierte Informationen wie Kanten oder Texturen aus Bildern, was besonders für die Klassifikation von Blattkrankheiten nützlich ist.  
Sie reduziert dabei die Datenmenge und hebt relevante Muster hervor – oft mit besserer Modellleistung bei geringerem Rechenaufwand als bei Rohpixeln.

Die Funktion w2d führt eine Haar-Wavelet-Transformation auf einem Bild durch und extrahiert gezielt Kanten- und Texturinformationen.

In [ ]:
def w2d(img, mode='haar', level=1):
    """
    Führt eine Wavelet-Transformation auf einem RGB-Bild durch.
    Gibt ein uint8-Graustufenbild mit extrahierten Texturinformationen zurück.
    """
    # In Graustufen umwandeln, falls RGB
    if len(img.shape) == 3:
        img_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    else:
        img_gray = img.copy()

    # In float konvertieren und normalisieren
    img_gray = np.float32(img_gray) / 255.0

    # Wavelet-Dekomposition
    coeffs = pywt.wavedec2(img_gray, mode, level=level)

    # Nur die Detailkoeffizienten verwenden
    coeffs_H = list(coeffs)
    coeffs_H[0] *= 0  # Approximationskoeffizienten auf 0 setzen

    # Rekonstruktion der Details
    img_reconstructed = pywt.waverec2(coeffs_H, mode)
    img_reconstructed = np.clip(img_reconstructed * 255, 0, 255).astype(np.uint8)

    return img_reconstructed


Zur Vorbereitung wurden alle Bilder einheitlich skaliert und in zwei Varianten verarbeitet:

- **Graustufen**: Reduktion der Farbkanäle für einfachere Merkmalsextraktion.
- **Wavelet-Transformation**: Extraktion von Textur- und Kanteninformationen.

Die verarbeiteten Bilder wurden in passenden Ordnern gespeichert und später zu flachen Feature-Vektoren umgewandelt.

In [ ]:
def convert_images(source_root=global_target_dir, image_size=global_image_size, 
                   processing_type='grayscale', mode='haar', level=global_w2d_level):
    total_images = 0

    # Prüfen, ob Quelldaten vorhanden sind
    if not os.path.exists(source_root):
        print("Datenordner existiert nicht!")
        return 
    
    # Zielordnername je nach Verarbeitungsart setzen
    output_folder = 'grayscale_features' if processing_type == 'grayscale' else 'wavelet_features'
    
    for plant_dir in os.listdir(source_root):
        plant_path = os.path.join(source_root, plant_dir)
        raw_images_path = os.path.join(plant_path, 'raw_images')
        output_path = os.path.join(plant_path, output_folder)
        
        if not os.path.isdir(raw_images_path):
            continue
        
        for class_name in os.listdir(raw_images_path):
            class_input_path = os.path.join(raw_images_path, class_name)
            class_output_path = os.path.join(output_path, class_name)
            
            # Zielordner neu erstellen (alte ggf. löschen)
            if os.path.exists(class_output_path):
                shutil.rmtree(class_output_path)
            os.makedirs(class_output_path)
            
            image_count = 0
            
            for filename in os.listdir(class_input_path):
                if filename.lower().endswith('.jpg'):
                    input_path = os.path.join(class_input_path, filename)
                    
                    try:
                        if processing_type == 'grayscale':
                            # Bild in Graustufen konvertieren und skalieren
                            img = Image.open(input_path).convert('L')
                            img = img.resize(image_size)
                            
                        elif processing_type == 'wavelet':
                            # Bild laden, auf RGB umwandeln, skalieren und Wavelet-Transformation anwenden
                            img = cv2.imread(input_path)
                            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                            img_rgb = cv2.resize(img_rgb, image_size)
                            img = w2d(img_rgb, mode=mode, level=level)
                        
                        # Verarbeitetes Bild speichern
                        base_name = os.path.splitext(filename)[0]
                        save_path = os.path.join(class_output_path, f"{base_name}.jpg")
                        
                        if processing_type == 'grayscale':
                            img.save(save_path)
                        else:
                            cv2.imwrite(save_path, img)
                        
                        image_count += 1
                    
                    except Exception as e:
                        print(f"Fehler bei {filename}: {e}")
            
            print(f"Kategorie '{class_name}': {image_count} Bilder nach '{class_output_path}' kopiert.")
            total_images += image_count
    
    print(f"\nTotal kopierte Bilder: {total_images}")
    print(f"Verarbeitung abgeschlossen: {processing_type.upper()}-Features gespeichert.")

 # Grayscale Konvertierung:
convert_images(processing_type='grayscale')

# Wavelet Konvertierung
convert_images(processing_type='wavelet', mode='haar', level=5)


***Erstellen von `X` und `Y` für klassische ML-Modelle und ANN***

Um klassische ML-Modelle oder ein einfaches ANN zu trainieren, müssen die Bilddaten in eine **tabellarische Form** gebracht werden.  
Dazu werden die Bilder aus den vorbereiteten **Graustufen- oder Wavelet-Verzeichnissen** geladen, auf eine einheitliche Größe gebracht und anschließend zu **flachen Vektoren (Feature-Vektoren)** umgewandelt.

- **`X_gray`** enthält die Graustufenbilder als flache Arrays(64 × 64 = 4096  Merkmale pro Bild).
- **`X_wavelet`** enthält die Wavelet-basierten Features ebenfalls als flache Arrays.
- **`y_full_gray`**  bzw. **`y_full_wavelet`** enthalten die zugehörigen Labelstrings in der Form: "Tomato___Early_blight", "Apple___Scab" usw.

Diese strukturierte Vorbereitung ist essenziell für alle Modelle, die keine Rohbilder (wie CNNs) direkt verarbeiten können.


In [ ]:
def load_flat_images_from(
    root_dir, 
    feature_dirname='grayscale_features', 
    return_paths=False
):
    X = []       # Bilddaten (flach)
    y = []       # Labels
    paths = []   # Optional: Pfade zu den Bildern

    for plant in os.listdir(root_dir):
        plant_path = os.path.join(root_dir, plant)
        feature_path = os.path.join(plant_path, feature_dirname)

        if not os.path.isdir(feature_path):
            continue

        print(f" Verarbeitung: {feature_path}")

        for class_name in os.listdir(feature_path):
            class_dir = os.path.join(feature_path, class_name)
            if not os.path.isdir(class_dir):
                continue

            for fname in os.listdir(class_dir):
                if fname.lower().endswith('.jpg'):
                    fpath = os.path.join(class_dir, fname)

                    try:
                        # Bild laden und in 1D-Array (flach) umwandeln
                        img = Image.open(fpath)
                        img_arr = np.array(img).flatten()
                        X.append(img_arr)

                        # Label aus Ordnerstruktur ableiten
                        y.append(class_name)

                        # Optional: Bildpfad speichern
                        if return_paths:
                            paths.append(fpath)

                    except Exception as e:
                        print(f" Fehler bei {fpath}: {e}")

    # Rückgabe abhängig von return_paths
    if return_paths:
        return np.array(X), y, paths
    else:
        return np.array(X), y

# Daten aus beiden Feature-Sätzen laden
X_gray, y_full_gray, y_path_gray = load_flat_images_from(global_target_dir, feature_dirname='grayscale_features', return_paths=True)
X_wavelet, y_full_wavelet, y_path_wavelet = load_flat_images_from(global_target_dir, feature_dirname='wavelet_features', return_paths=True)

Die Datenvorverarbeitung ist abgeschlossen. Alle Bilder wurden auf eine einheitliche Größe von 64 × 64 Pixeln gebracht, in Graustufen umgewandelt und zusätzlich einer Wavelet-Transformation unterzogen. Dadurch entstanden zwei parallele Datensätze, die als Eingabe für verschiedene Modelle genutzt werden können.

- **`X_gray`** : Flache Vektoren aus Graustufenbildern,Shape: (11861, 4096)
- **`X_wavelet`** : Flache Vektoren aus Wavelet-transformierten Bildern, Shape: (11861, 4096)
- **`y_full_gray`** , **`y_full_wavelet`** : Labels im Format "Pflanze___Zustand"

Zur Kontrolle wird hier ein zufällig gewählter Datensatzindex visualisiert: Das Originalbild, das Graustufenbild und das resultierende Wavelet-Bild mit der zugehörigen Beschriftung:

In [ ]:
i = 42  # beliebiger Index
fig, axs = plt.subplots(1, 3, figsize=(12,4))

# Originalbild
raw_path = y_path_gray[i].replace('grayscale_features', 'raw_images')
img_raw = Image.open(raw_path)
axs[0].imshow(img_raw)
axs[0].set_title("Original")

# Graustufenbild
axs[1].imshow(X_gray[i].reshape(*global_image_size), cmap='gray')
axs[1].set_title("Graustufen")

# Waveletbild
axs[2].imshow(X_wavelet[i].reshape(*global_image_size), cmap='gray')
axs[2].set_title("Wavelet")

for ax in axs:
    ax.axis('off')

plt.suptitle(y_full_gray[i])
plt.tight_layout()
plt.show()

##  Modellierung / Model Building

In [ ]:
def filter_multiple_plants(X, Y, plant_names):
    plant_data = {}  # Ergebnis-Dictionary: enthält pro Pflanze X und y

    for plant_name in plant_names:
        X_filtered = []     # Gefilterte Bilddaten für diese Pflanze
        y_filtered = []     # Gefilterte Labels (nur Krankheitszustand)
        count = 0           # Zähler für gefundene Bilder

        for xi, label in zip(X, Y):
            try:
                parts = label.split("___")
                if len(parts) != 2:
                    continue

                plant, _ = parts

                # Nur Bilder dieser Pflanze berücksichtigen
                if plant == plant_name:
                    X_filtered.append(xi)
                    y_filtered.append(label)
                    count += 1
            except:
                continue  # Fehlerhafte Einträge überspringen

        print(f" '{plant_name}': {count} Bilder gefunden.")
        plant_data[plant_name] = (np.array(X_filtered), y_filtered)

    return plant_data


plant_names = ['Apple', 'Corn', 'Tomato']

# Datensätze für jede Pflanze getrennt laden
filtered_data_gray = filter_multiple_plants(X_gray, y_full_gray, plant_names)
filtered_data_wavelet = filter_multiple_plants(X_wavelet, y_full_wavelet, plant_names)

In [ ]:
def prepareDataToModel(filtered_data, categorie):
    X_data, Y_data = filtered_data[categorie]
    X = X_data
    y = Y_data

    # Label-Encoding der Zielklassen (z. B. 'Apple___healthy' → 0, ...)
    le = LabelEncoder()
    y_encoded = le.fit_transform(y)

    ''' Aufteilen in Trainings- und Testdaten (80/20)
        Die Daten werden automatisch geshuffelt (random_state für Reproduzierbarkeit).
        Mit stratify=y_encoded bleibt die Klassenverteilung im Train-/Testset erhalten.
        wichtig bei unbalancierten Klassen (z. B. 'Apple___healthy' deutlich häufiger)
    '''
    X_train, X_test, y_train, y_test = train_test_split(
        X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
    )

    return X_train, X_test, y_train, y_test

# Beispielhafte Vorbereitung der Apple-Daten (Gray + Wavelet)
X_train_gray, X_test_gray, y_train_gray, y_test_gray = prepareDataToModel(filtered_data_gray, 'Apple')
X_train_wavelet, X_test_wavelet, y_train_wavelet, y_test_wavelet = prepareDataToModel(filtered_data_wavelet, 'Apple')

In [ ]:
'''
ML-Modelle mit zugehörigen Hyperparametern für GridSearchCV:
- SVM (linear)
- Random Forest (Ensemble aus Decision Trees)
- Decision Tree (einzelner Baum)
- Logistische Regression

Obwohl Random Forest auf Decision Trees basiert, wurden beide separat verglichen,
um Einfachmodell vs. Ensemble gegenüberzustellen.
'''

model_params = {
    'svm': {
        'model': SVC(gamma='auto', probability=True),
        'params': {
            'svc__C': [0.1, 1, 10],
            'svc__kernel': ['linear'] 
        }
    },
    'random_forest': {
        'model': RandomForestClassifier(n_jobs=-1),
        'params': {
            'randomforestclassifier__n_estimators': [100, 200, 300],
            'randomforestclassifier__max_features': ['sqrt', 'log2']
        }
    },
    'decision_tree': {
        'model': DecisionTreeClassifier(),
        'params': {
            'decisiontreeclassifier__max_depth': [10, 20, None],
            'decisiontreeclassifier__min_samples_split': [2, 5, 10]
        }
    },
    'logistic_regression': {
        'model': LogisticRegression(solver='liblinear', multi_class='auto', max_iter=1000),
        'params': {
            'logisticregression__C': [0.01, 0.1, 1]
        }
    }
}

### Separate Klassifikation (Ansatz A)

**Appfel**

***Machine Lerning***

In [ ]:
def gridsearch_multiple_models(X_train, y_train,model_params,cv=3): 
    scores = []
    best_estimators = {}

    for algo, mp in model_params.items():
        pipe = make_pipeline(StandardScaler(), mp['model'])
        grid = GridSearchCV(pipe, mp['params'], cv=cv, n_jobs=-1,return_train_score=False)
        grid.fit(X_train, y_train)
        scores.append({
            'model': algo,
            'best_score': grid.best_score_,
            'best_params': grid.best_params_
        })
        best_estimators[algo] = grid.best_estimator_

    df = pd.DataFrame(scores, columns=['model', 'best_score', 'best_params'])
    return df, best_estimators

Die `X_gray`-Daten wurden zur Modelloptimierung verwendet. Das leistungsstärkste Modell (basierend auf Cross-Validation) wurde ermittelt und anschließend zur späteren Wiederverwendung als `.pkl`-Datei gespeichert.

In [ ]:
# Speicherung des trainierten Modells (klassisch oder Deep Learning)
def save_model(model, path_directory, file_name, model_type='ml'):
    if not os.path.exists(path_directory):
        os.makedirs(path_directory)
    
    save_path = os.path.join(path_directory, file_name)
    
    if model_type == 'ml':
        joblib.dump(model, save_path)
    elif model_type == 'dl':
        model.save(save_path)
    else:
        raise ValueError("Unbekannter Modelltyp: 'ml' (scikit-learn) oder 'dl' (Deep Learning/Keras) erwartet.")

In [ ]:
# ML-Model Trainieren
df, estimators = gridsearch_multiple_models(X_train_gray, y_train_gray, model_params)

# Bestes Modell wählen
best_model_name = df.loc[df['best_score'].idxmax()]['model']
best_model = estimators[best_model_name]

# Trainiertes Modell speichern
save_model(best_model,'saved_models/v1/ml/apple','best_model_apple_gray_new.pkl')

In [ ]:
# ML-Model Trainieren
df, estimators = gridsearch_multiple_models(X_train_wavelet, y_train_wavelet,model_params)

# Bestes Modell wählen
best_model_name = df.loc[df['best_score'].idxmax()]['model']
best_model = estimators[best_model_name]

# Speichern
save_model(best_model,'saved_models/v1/ml/apple','best_model_apple_wavelet.pkl')

***Künstliches Neuronales Netz (ANN)***

Für jede Pflanzenkategorie wurde ein Artificial Neural Network (ANN) erstellt, das auf flach dargestellten Bildmerkmalen basiert (`X_gray`, `X_wavelet`).  

In [ ]:
def get_optimal_batch_size(dataset_size, model_type='ann'):

    if model_type.lower() == 'cnn':
        if dataset_size <= 3000:
            return 32
        elif dataset_size <= 6000:
            return 48
        elif dataset_size <= 9000:
            return 64
        else: 
            return 80
    
    else: 
        if dataset_size <= 3000:
            return 48
        elif dataset_size <= 6000:
            return 64
        elif dataset_size <= 9000:
            return 96
        else:
            return 128

def train_model(model, X_train, y_train, X_test, y_test, model_type='ann'):
    
    # Ermittele die optimale Batch-Größe
    dataset_size = len(X_train)
    optimal_batch_size = get_optimal_batch_size(dataset_size, model_type)
    
    print(f"Datensatzgröße: {dataset_size}")
    print(f"Modelltyp: {model_type.upper()}")
    print(f"Optimale Batch-Größe: {optimal_batch_size}")
    
    callbacks = [
        EarlyStopping(
            patience=15,  
            restore_best_weights=True,
            verbose=1,
            monitor='val_accuracy' 
        ),
        ReduceLROnPlateau(
            factor=0.7, 
            patience=8,
            min_lr=1e-6,
            verbose=1
        )
    ]
    
    # Modell kompilieren
    model.compile(
        optimizer=AdamW(learning_rate=0.0005),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    history = model.fit(
        X_train, y_train,
        epochs=150, 
        batch_size=optimal_batch_size, 
        validation_data=(X_test, y_test),
        callbacks=callbacks,
        verbose=1
    )
    
    return history

def create_ann(output=3):
    """ANN model architecture"""
    model = models.Sequential([
        layers.Input(shape=(global_image_size[0]*global_image_size[1],)),
        
        # Mehr Kapazität für komplexe Muster
        layers.Dense(1024, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.4),
        
        layers.Dense(512, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.3),
        
        layers.Dense(256, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.2),
        
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.1),
        
        layers.Dense(output, activation='softmax')
    ])
    return model

In [ ]:
def cross_validate_ann_adaptive(X, y, create_model_fn, model_type='ann', cv_folds=3, random_state=42):

    # KFold-Cross-Validation mit Shuffling und fixiertem random_state für Reproduzierbarkeit
    kf = KFold(n_splits=cv_folds, shuffle=True, random_state=random_state)
    val_accuracies = []

    for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
        print(f"\n Fold {fold+1}/{cv_folds}")

        # Trainings- und Validierungsdaten für aktuellen Fold aufteilen
        X_train_fold, X_val_fold = X[train_idx], X[val_idx]
        y_train_fold, y_val_fold = y[train_idx], y[val_idx]

        # Feature-Skalierung (only for ANN, CNNs typically don't need it)
        if model_type.lower() == 'ann':
            scaler = StandardScaler()
            X_train_fold_scaled = scaler.fit_transform(X_train_fold)
            X_val_fold_scaled = scaler.transform(X_val_fold)
        else:
            X_train_fold_scaled = X_train_fold
            X_val_fold_scaled = X_val_fold

        # Neues Modell erstellen
        model = create_model_fn()

        # Modelltraining mit adaptiver Batch-Größe
        train_model(
            model,
            X_train_fold_scaled, y_train_fold,
            X_val_fold_scaled, y_val_fold,
            model_type=model_type
        )

        # Validierungsgenauigkeit messen
        val_loss, val_accuracy = model.evaluate(X_val_fold_scaled, y_val_fold, verbose=0)
        print(f" Fold-{fold+1} Validierungs-Genauigkeit: {val_accuracy:.4f}")
        val_accuracies.append(val_accuracy)

    # Durchschnittliche Genauigkeit über alle Folds
    mean_val_acc = np.mean(val_accuracies)
    print(f"\n Durchschnittliche CV-Genauigkeit über {cv_folds} Folds: {mean_val_acc:.4f}")

    return mean_val_acc

***Cross-Validation Gray Data*** 

In [ ]:
## Cross-Validation für das ANN-Modell mit 3 Folds
mean_accuracy = cross_validate_ann_adaptive(
    X_train_gray, y_train_gray,
    create_model_fn=create_ann,
    model_type='ann', 
    cv_folds=3
)

In [ ]:
## Funktion zur Skalierung von Trainings- und Testdaten
def scale_data(X_train, X_test):
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    return X_train_scaled, X_test_scaled

In [ ]:
# Daten skalieren (Standardisierung)
X_train_gray_scaled, X_test_gray_scaled = scale_data(X_train_gray, X_test_gray)

# Löscht vorherige Modelle aus dem Speicher und verhindert Speicherüberlauf bei mehrfacher Modellerstellung
K.clear_session()

# ANN-Modell erstellen und trainieren
model = create_ann() 
history = train_model(model, X_train_gray_scaled, y_train_gray, X_test_gray_scaled, y_test_gray)

# Trainiertes Modell speichern (Deep Learning Format)
save_model(model, 'saved_models/v1/ann/apple', 'model_apple_gray.keras', 'dl')

***Cross-Validation Wavelet Data*** 

In [ ]:
mean_accuracy = cross_validate_ann_adaptive(
    X_train_wavelet, y_train_wavelet,
    create_model_fn=create_ann,
    model_type='ann', 
    cv_folds=3
)

In [ ]:
# Daten skalieren (Standardisierung)
X_train_wavelet_scaled,X_test_wavelet_scaled = scale_data(X_train_wavelet, X_test_wavelet)

# Löscht vorherige Modelle aus dem Speicher
K.clear_session()

# Modellinitialisierung und Training
model = create_ann() 
history = train_model(model, X_train_wavelet_scaled, y_train_wavelet, X_test_wavelet_scaled, y_test_wavelet)
    
# Speicherung des trainierten Modells
save_model(model,'saved_models/v1/ann/apple','model_apple_wavelet.keras','dl')

***Faltungsneuronales Netz (CNN)***

In [ ]:
# Datenvorbereitung für CNN
def prepare_cnn_input(X_train, X_test, image_size):
    X_train_cnn = X_train.reshape(-1, image_size[0], image_size[1], 1) / 255.0
    X_test_cnn = X_test.reshape(-1, image_size[0], image_size[1], 1) / 255.0
    return X_train_cnn, X_test_cnn

# CNN-Eingabe: Von flachen Vektoren zu 4D-Tensoren (und normalisieren)
X_train_gray_cnn, X_test_gray_cnn = prepare_cnn_input(X_train_gray, X_test_gray, global_image_size)
X_train_wavelet_cnn, X_test_wavelet_cnn = prepare_cnn_input(X_train_wavelet, X_test_wavelet, global_image_size)

In [ ]:
def create_cnn_model(output_base=3, use_augmentation=False):
    model = models.Sequential()


    model.add(layers.Input(shape=(global_image_size[0], global_image_size[1], 1)))

    if use_augmentation:
        data_augmentation = tf.keras.Sequential([
            layers.RandomFlip("horizontal"),
            layers.RandomRotation(0.1),
            layers.RandomZoom(0.1),
            layers.RandomContrast(0.1)
        ])
        model.add(data_augmentation)

    # Convolutional Layers
    model.add(layers.Conv2D(32, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.BatchNormalization())

    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.BatchNormalization())

    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.BatchNormalization())

    model.add(layers.Conv2D(256, (3, 3), activation='relu'))
    model.add(layers.GlobalAveragePooling2D())

    # Dense Layers
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(output_base, activation='softmax'))

    return model

Das CNN-Modell besteht aus vier Faltungsblöcken mit jeweils Conv2D, MaxPooling und Batch-Normalisierung, gefolgt von dichten Schichten mit Dropout zur Vermeidung von Overfitting und endet mit einer Softmax-Ausgabe für die dreiklassige Klassifikation.


***Training des CNN-Modells ohne Augmentierung***

In [ ]:
# Modellinitialisierung und Training
model = create_cnn_model() 
history = train_model(model, X_train_gray_cnn, y_train_gray, X_test_gray_cnn, y_test_gray,model_type='cnn')

# Löscht vorherige Modelle aus dem Speicher
K.clear_session()

# Speicherung des trainierten Modells
save_model(model,'saved_models/v1/cnn/apple','model_without_augmentation_apple_gray.keras','dl')

K.clear_session()

# Modellinitialisierung und Training
model = create_cnn_model() 
history = train_model(model, X_train_wavelet_cnn, y_train_wavelet, X_test_wavelet_cnn, y_test_wavelet,model_type='cnn')

# Speicherung des trainierten Modells
save_model(model,'saved_models/v1/cnn/apple','model_without_augmentation_apple_wavelet.keras','dl')

***Training des CNN-Modells mit Augmentierung***

In [ ]:
K.clear_session()

# Modellinitialisierung und Training
model = create_cnn_model(output_base=3, use_augmentation=True)
history = train_model(model, X_train_gray_cnn, y_train_gray, X_test_gray_cnn, y_test_gray,model_type='cnn')

# Speicherung des trainierten Modells
save_model(model,'saved_models/v1/cnn/apple','model_with_augmentation_apple_gray.keras','dl')
K.clear_session()

# Modellinitialisierung und Training
model = create_cnn_model(output_base=3, use_augmentation=True)
history = train_model(model, X_train_wavelet_cnn, y_train_wavelet, X_test_wavelet_cnn, y_test_wavelet,model_type='cnn')

# Speicherung des trainierten Modells
save_model(model,'saved_models/v1/cnn/apple','model_with_augmentation_apple_wavelet.keras','dl')

**Tomaten**

***Machine Learning***

In [ ]:
## Data für Tomaten vorbereiten
X_train_gray, X_test_gray, y_train_gray, y_test_gray=prepareDataToModel(filtered_data_gray, 'Tomato')  
X_train_wavelet, X_test_wavelet, y_train_wavelet, y_test_wavelet=prepareDataToModel(filtered_data_wavelet, 'Tomato') 

In [ ]:
# ML-Model Trainieren
df, estimators = gridsearch_multiple_models(X_train_gray, y_train_gray, model_params)

# Bestes Modell wählen
best_model_name = df.loc[df['best_score'].idxmax()]['model']
best_model = estimators[best_model_name]

# Speicherung des trainierten Modells
save_model(best_model, f'saved_models/v1/ml/tomato','best_model_tomato_gray.pkl')

# ML-Model Trainieren
df, estimators = gridsearch_multiple_models(X_train_wavelet, y_train_wavelet,model_params)

# Bestes Modell wählen
best_model_name = df.loc[df['best_score'].idxmax()]['model']
best_model = estimators[best_model_name]

# Speicherung des trainierten Modells
save_model(best_model, f'saved_models/v1/ml/tomato','best_model_tomato_wavelet.pkl')

***Künstliches Neuronales Netz (ANN)***

***Cross Validation Gray Data***

In [ ]:
# Cross-Validation für das ANN-Modell mit 3 Folds
mean_accuracy = cross_validate_ann_adaptive(
    X_train_gray, y_train_gray,
    create_model_fn=create_ann,
    model_type='ann', 
    cv_folds=3
)

In [ ]:
# Daten skalieren (Standardisierung)
X_train_gray_scaled, X_test_gray_scaled  = scale_data(X_train_gray, X_test_gray)
K.clear_session()

# Modellinitialisierung und Training
model = create_ann() 
history = train_model(model, X_train_gray_scaled, y_train_gray, X_test_gray_scaled, y_test_gray)

# Speicherung des trainierten Modells
save_model(model,'saved_models/v1/ann/tomato','model_tomato_gray.keras','dl')    

***Cross Validation Wavelet Data***

In [ ]:
mean_accuracy = cross_validate_ann_adaptive(
    X_train_wavelet, y_train_wavelet,
    create_model_fn=create_ann,
    model_type='ann', 
    cv_folds=3
)

In [ ]:
# Daten skalieren (Standardisierung)
X_train_wavelet_scaled,X_test_wavelet_scaled = scale_data(X_train_wavelet, X_test_wavelet)
K.clear_session()

# Modellinitialisierung und Training
model = create_ann() 
history = train_model(model, X_train_wavelet_scaled, y_train_wavelet, X_test_wavelet_scaled, y_test_wavelet)

# Speicherung des trainierten Modells
save_model(model,'saved_models/v1/ann/tomato','model_tomato_wavelet.keras','dl')

***Faltungsneuronales Netz (CNN)***

In [ ]:
# CNN-Eingabe: Von flachen Vektoren zu 4D-Tensoren (und normalisieren)
X_train_gray_cnn, X_test_gray_cnn = prepare_cnn_input(X_train_gray, X_test_gray, global_image_size)
X_train_wavelet_cnn, X_test_wavelet_cnn = prepare_cnn_input(X_train_wavelet, X_test_wavelet, global_image_size)

***Training des CNN-Modells ohne Augmentierung***

In [ ]:
K.clear_session()

# Modellinitialisierung und Training
model = create_cnn_model() 
history = train_model(model, X_train_gray_cnn, y_train_gray, X_test_gray_cnn, y_test_gray,model_type='cnn')

# Speicherung des trainierten Modells
save_model(model,'saved_models/v1/cnn/tomato','model_without_augmentation_tomato_gray.keras','dl')

K.clear_session()

# Modellinitialisierung und Training
model = create_cnn_model() 
history = train_model(model, X_train_wavelet_cnn, y_train_wavelet, X_test_wavelet_cnn, y_test_wavelet,model_type='cnn')

# Speicherung des trainierten Modells
save_model(model,'saved_models/v1/cnn/tomato','model_without_augmentation_tomato_wavelet.keras','dl')

***Training des CNN-Modells mit Augmentierung***

In [ ]:
K.clear_session()

# Modellinitialisierung und Training
model = create_cnn_model(output_base=3, use_augmentation=True)
history = train_model(model, X_train_gray_cnn, y_train_gray, X_test_gray_cnn, y_test_gray,model_type='cnn')

# Speicherung des trainierten Modells
save_model(model,'saved_models/v1/cnn/tomato','model_with_augmentation_tomato_gray.keras','dl')
K.clear_session()

# Modellinitialisierung und Training
model = create_cnn_model(output_base=3, use_augmentation=True)
history = train_model(model, X_train_wavelet_cnn, y_train_wavelet, X_test_wavelet_cnn, y_test_wavelet,model_type='cnn')

# Speicherung des trainierten Modells
save_model(model,'saved_models/v1/cnn/tomato','model_with_augmentation_tomato_wavelet.keras','dl')

**Mais**

***Machine Learning***

In [ ]:
X_train_gray, X_test_gray, y_train_gray, y_test_gray=prepareDataToModel(filtered_data_gray, 'Corn')  
X_train_wavelet, X_test_wavelet, y_train_wavelet, y_test_wavelet=prepareDataToModel(filtered_data_wavelet, 'Corn') 

In [ ]:
# ML-Model Trainieren
df, estimators = gridsearch_multiple_models(X_train_gray, y_train_gray, model_params)

# Bestes Modell wählen
best_model_name = df.loc[df['best_score'].idxmax()]['model']
best_model = estimators[best_model_name]

# Speicherung des trainierten Modells
save_model(best_model, f'saved_models/v1/ml/corn','best_model_corn_gray.pkl')

# ML-Model Trainieren
df, estimators = gridsearch_multiple_models(X_train_wavelet, y_train_wavelet,model_params)

# Bestes Modell wählen
best_model_name = df.loc[df['best_score'].idxmax()]['model']
print(best_model_name)
best_model = estimators[best_model_name]

# Speicherung des trainierten Modells
save_model(best_model, f'saved_models/v1/ml/corn','best_model_corn_wavelet.pkl')

***Künstliches Neuronales Netz (ANN)***

***Cross Validation Gray Data***

In [ ]:
mean_accuracy = cross_validate_ann_adaptive(
    X_train_gray, y_train_gray,
    create_model_fn=create_ann,
    model_type='ann', 
    cv_folds=3
)

In [ ]:
# Daten skalieren (Standardisierung)
X_train_gray_scaled, X_test_gray_scaled  = scale_data(X_train_gray, X_test_gray)

K.clear_session()

# Modellinitialisierung und Training
model = create_ann() 
history = train_model(model, X_train_gray_scaled, y_train_gray, X_test_gray_scaled, y_test_gray)
    
# Speicherung des trainierten Modells
save_model(model,'saved_models/v1/ann/corn','model_corn_gray.keras','dl')

***Cross Validation Wavelet Data***

In [ ]:
mean_accuracy = cross_validate_ann_adaptive(
    X_train_wavelet, y_train_wavelet,
    create_model_fn=create_ann,
    model_type='ann', 
    cv_folds=3
)

In [ ]:
# Daten skalieren (Standardisierung)
X_train_wavelet_scaled,X_test_wavelet_scaled = scale_data(X_train_wavelet, X_test_wavelet)

K.clear_session()

# Modellinitialisierung und Training
model = create_ann() 
history = train_model(model, X_train_wavelet_scaled, y_train_wavelet, X_test_wavelet_scaled, y_test_wavelet)
    
# Speicherung des trainierten Modells
save_model(model,'saved_models/v1/ann/corn','model_corn_wavelet.keras','dl')

***Training des CNN-Modells ohne Augmentierung***

In [ ]:
# Data vorbereiten
X_train_gray_cnn, X_test_gray_cnn = prepare_cnn_input(X_train_gray, X_test_gray, global_image_size)
X_train_wavelet_cnn, X_test_wavelet_cnn = prepare_cnn_input(X_train_wavelet, X_test_wavelet, global_image_size)


K.clear_session()

# Modellinitialisierung und Training
model = create_cnn_model() 
history = train_model(model, X_train_gray_cnn, y_train_gray, X_test_gray_cnn, y_test_gray,model_type='cnn')

# Speicherung des trainierten Modells
save_model(model,'saved_models/v1/cnn/corn','model_without_augmentation_corn_gray.keras','dl')

K.clear_session()

# Modellinitialisierung und Training
model = create_cnn_model() 
history = train_model(model, X_train_wavelet_cnn, y_train_wavelet, X_test_wavelet_cnn, y_test_wavelet,model_type='cnn')

# Speicherung des trainierten Modells
save_model(model,'saved_models/v1/cnn/corn','model_without_augmentation_corn_wavelet.keras','dl')

***Training des CNN-Modells mit Augmentierung***

In [ ]:
K.clear_session()

# Modellinitialisierung und Training
model = create_cnn_model(output_base=3, use_augmentation=True)
history = train_model(model, X_train_gray_cnn, y_train_gray, X_test_gray_cnn, y_test_gray,model_type='cnn')

# Speicherung des trainierten Modells
save_model(model,'saved_models/v1/cnn/corn','model_with_augmentation_corn_gray.keras','dl')

K.clear_session()

# Modellinitialisierung und Training
model = create_cnn_model(output_base=3, use_augmentation=True)
history = train_model(model, X_train_wavelet_cnn, y_train_wavelet, X_test_wavelet_cnn, y_test_wavelet,model_type='cnn')

# Speicherung des trainierten Modells
save_model(model,'saved_models/v1/cnn/corn','model_with_augmentation_corn_wavelet.keras','dl')

### Kombinierte Klassifikation (Ansatz B)

In [ ]:
def prepare_combined_data(X, y_full):
    le = LabelEncoder()
    y_encoded = le.fit_transform(y_full)

    ''' Aufteilen in Trainings- und Testdaten (80/20)
        Die Daten werden automatisch geshuffelt (random_state für Reproduzierbarkeit).
        Mit stratify=y_encoded bleibt die Klassenverteilung im Train-/Testset erhalten.
        wichtig bei unbalancierten Klassen (z.B. 'Apple___healthy' deutlich häufiger)
    '''
    X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded )
    return X_train, X_test, y_train, y_test

X_train_gray, X_test_gray, y_train_gray, y_test_gray = prepare_combined_data(X_gray, y_full_gray)
X_train_wavelet, X_test_wavelet, y_train_wavelet, y_test_wavelet = prepare_combined_data(X_wavelet, y_full_wavelet)

In [ ]:
model_params = {
    'svm': {
        'model': SVC(gamma='auto', probability=True),
        'params': {
            'svc__C': [1],
            'svc__kernel': ['linear']  
        }
    },
    'random_forest': {
        'model': RandomForestClassifier(n_jobs=-1),
        'params': {
            'randomforestclassifier__n_estimators': [100], 
            'randomforestclassifier__max_features': ['sqrt']
        }
    },
    'decision_tree': {
        'model': DecisionTreeClassifier(),
        'params': {
            'decisiontreeclassifier__max_depth': [10],
            'decisiontreeclassifier__min_samples_split': [2]
        }
    },
    'logistic_regression': {
        'model': LogisticRegression(solver='liblinear', multi_class='auto', max_iter=1000),
        'params': {
            'logisticregression__C': [0.1]
        }
    }
}

***Machine Learning***

In [ ]:
# ML-Model Trainieren
df, estimators = gridsearch_multiple_models(X_train_gray, y_train_gray, model_params,2)

# Bestes Modell wählen 
best_model_name = df.loc[df['best_score'].idxmax()]['model']
best_model = estimators[best_model_name]

# Speicherung des trainierten Modells
save_model(best_model,'saved_models/v2/ml','best_model_gray.pkl')

df, estimators = gridsearch_multiple_models(X_train_wavelet, y_train_wavelet, model_params,2)

# Bestes Modell wählen
best_model_name = df.loc[df['best_score'].idxmax()]['model']
best_model = estimators[best_model_name]

# Speicherung des trainierten Modells
save_model(best_model,'saved_models/v2/ml','best_model_wavelet.pkl')

***Künstliches Neuronales Netz (ANN)***

In [ ]:
# Skalierung, Modellinitialisierung und Training
X_train_gray_scaled,X_test_gray_scaled = scale_data(X_train_gray,X_test_gray)

K.clear_session()

# Modellinitialisierung und Training
model = create_ann(9) 
history = train_model(model, X_train_gray_scaled, y_train_gray, X_test_gray_scaled, y_test_gray)

# Speicherung des trainierten Modells
save_model(model,'saved_models/v2/ann','model_gray.keras','dl')

# Skalierung, Modellinitialisierung und Training
X_train_wavelet_scaled , X_test_wavelet_scaled= scale_data(X_train_wavelet,X_test_wavelet)

K.clear_session()

# Modellinitialisierung und Training
model = create_ann(9) 
history = train_model(model, X_train_wavelet_scaled, y_train_wavelet, X_test_wavelet_scaled, y_test_wavelet)

# Speicherung des trainierten Modells
save_model(model,'saved_models/v2/ann','model_wavelet.keras','dl')

***Faltungsneuronales Netz (CNN)***

In [ ]:
# CNN-Eingabe: Von flachen Vektoren zu 4D-Tensoren (und normalisieren)
X_train_gray_cnn, X_test_gray_cnn = prepare_cnn_input(X_train_gray, X_test_gray, global_image_size)
X_train_wavelet_cnn, X_test_wavelet_cnn = prepare_cnn_input(X_train_wavelet, X_test_wavelet, global_image_size)

***Training des CNN-Modells ohne Augmentierung***

In [ ]:
K.clear_session()

# Modellinitialisierung und Training
model = create_cnn_model(9) 
history = train_model(model, X_train_gray_cnn, y_train_gray, X_test_gray_cnn, y_test_gray,model_type='cnn')

# Speicherung des trainierten Modells
save_model(model,'saved_models/v2/cnn','model_without_augmentation_gray.keras','dl')

K.clear_session()

# Modellinitialisierung und Training
model = create_cnn_model(9) 
history = train_model(model, X_train_wavelet_cnn, y_train_wavelet, X_test_wavelet_cnn, y_test_wavelet,model_type='cnn')

# Speicherung des trainierten Modells
save_model(model,'saved_models/v2/cnn','model_without_augmentation_wavelet.keras','dl')

***Training des CNN-Modells mit Augmentierung***

In [ ]:
K.clear_session()

# Modellinitialisierung und Training
model = create_cnn_model(9,use_augmentation=True) 
history = train_model(model, X_train_gray_cnn, y_train_gray, X_test_gray_cnn, y_test_gray,model_type='cnn')

# Speicherung des trainierten Modells
save_model(model,'saved_models/v2/cnn','model_with_augmentation_gray.keras','dl')

K.clear_session()

# Modellinitialisierung und Training
model = create_cnn_model(9,use_augmentation=True) 
history = train_model(model, X_train_wavelet_cnn, y_train_wavelet, X_test_wavelet_cnn, y_test_wavelet,model_type='cnn')

# Speicherung des trainierten Modells
save_model(model,'saved_models/v2/cnn','model_with_augmentation_wavelet.keras','dl')

## Evaluation und Vergleich

### 4.1 Bewertungsmethoden

Zur Bewertung der Klassifikationsmodelle werden folgende Metriken verwendet:

- **Accuracy**: Anteil korrekt vorhergesagter Klassen.
- **Confusion Matrix**: Detaillierte Übersicht über falsch und richtig klassifizierte Klassen.
- **F1-Score**, **Precision** und **Recall**: Ergänzende Kennzahlen für die differenzierte Bewertung bei mehrklassigen oder unausgewogenen Datensätzen.

***Ergebnisse – Ansatz A: Separate Klassifikation***

In [ ]:
# Evaluation
def evaluate_model(model,model_type, category_name,data_Variant, X_test, y_test):

    y_pred = model.predict(X_test)

    if y_pred.ndim > 1 and y_pred.shape[1] > 1:
        y_pred = np.argmax(y_pred, axis=1)
    
    acc = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred, output_dict=True)
    
    result = {
        category_name: {
        'Model': model_type,
        'Category': category_name,  
        'Data_Variant': data_Variant,  
        'Accuracy': round(acc, 4),
        'Precision': round(report['macro avg']['precision'], 4),
        'Recall': round(report['macro avg']['recall'], 4),
        'F1-Score': round(report['macro avg']['f1-score'], 4),
        }
    }
    return result[category_name]
def evualateAllModels(categories, data_variants, model_types, version='v1'):
    results = []

    for category in categories:
        for model_type in model_types:
            for variant in data_variants:
                print(f"{model_type.upper()} | {category} | {variant} | Version: {version}")

                base_path = f"saved_models/{version}/{model_type}/{category.lower()}/"
                
                models_to_evaluate = []

                # ML-Modelle laden
                if model_type == 'ml':
                    model_path =   f"{base_path}best_model_{category}_{variant}.pkl"
        
                    try:
                        model = joblib.load(model_path)
                        models_to_evaluate.append((f"ML - {variant}", model))
                    except Exception as e:
                        print(f"ML-Modell nicht gefunden: {model_path}")

                elif model_type == 'ann':
                    model_path =      f"{base_path}model_{category}_{variant}.keras"
                    try:
                        model = load_model(model_path)
                        models_to_evaluate.append((f"ANN - {variant}", model))
                    except Exception as e:
                        print(f"ANN-Modell nicht gefunden: {model_path}")

                # CNN-Modelle laden (mit & ohne Augmentierung)
                elif model_type == 'cnn':
                  
                        model_path_no_aug = f"{base_path}model_without_augmentation_{category}_{variant}.keras"
                        try:
                            model = load_model(model_path_no_aug)
                            models_to_evaluate.append((f"CNN (ohne Aug.) - {variant}", model))
                        except Exception as e:
                             print(f" Kein Modell ohne Augmentierung gefunden: {model_path_no_aug}")

                        model_path_aug = f"{base_path}model_with_augmentation_{category}_{variant}.keras"
                        try:
                            model = load_model(model_path_aug)
                            models_to_evaluate.append((f"CNN (mit Aug.) - {variant}", model))
                        except Exception as e:
                           print(f" Kein Modell mit Augmentierung gefunden: {model_path_aug}")

                # Daten vorbereiten
                if variant == 'gray':
                    X_train, X_test, _, y_test = prepareDataToModel(filtered_data_gray, category.capitalize())
                else:
                    X_train, X_test, _, y_test = prepareDataToModel(filtered_data_wavelet, category.capitalize())

                # ANN: Skalieren
                if model_type == 'ann':
                    scaler = StandardScaler()
                    X_train = scaler.fit_transform(X_train)
                    X_test = scaler.transform(X_test)

                # CNN: Reshape & Normalisieren
                if model_type == 'cnn':
                    X_test = X_test.reshape(-1, global_image_size[0], global_image_size[1], 1) / 255.0

                # Modelle evaluieren
                for model_label, model in models_to_evaluate:
                    result = evaluate_model(model, model_label, category, variant.capitalize(), X_test, y_test)
                    results.append(result)

    return results

In [ ]:
categories = ['apple','tomato','corn']
data_variants = ['gray', 'wavelet']
model_types = ['ml', 'ann','cnn']

result=evualateAllModels(categories,data_variants,model_types,'v1')

df_results_v1 = pd.DataFrame(result)

pd.set_option('display.precision', 4)
df_results_v1.style.set_caption("Tabelle 4.1: Vergleich der Modelle für Variante A") \
                .set_table_styles([{
                    'selector': 'caption',
                    'props': [('color', 'red'), ('font-size', '15px')]
                }])

***Ergebnisse – Ansatz B: Kombinierte Klassifikation**

- ***Confusion Matrix***

In [ ]:
# Daten vorbereiten
X_train, X_test, y_train, y_test = prepare_combined_data(X_gray, y_full_gray)
X_test = X_test.reshape(-1, global_image_size[0], global_image_size[1], 1) / 255.0

# Vorgefertigtes Modell laden (hier CNN ohne Datenaugmentation)
model_path='saved_models/v2/cnn/model_without_augmentation_gray.keras'
model = load_model(model_path)

# Modellvorhersagen als Wahrscheinlichkeiten berechnen
y_pred_proba = model.predict(X_test)

# Für jede Probe die Klasse mit der höchsten Wahrscheinlichkeit auswählen
y_pred = y_pred_proba.argmax(axis=1)

# Klassenliste erstellen, sortiert und eindeutig, für Achsenbeschriftung der Confusion Matrix
labels = sorted(list(set(y_test)))

# Confusion Matrix berechnen
cm = confusion_matrix(y_test, y_pred, labels=labels)

print(cm)

# Confusion Matrix visualisieren
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)
fig, ax = plt.subplots(figsize=(8, 8))
disp.plot(ax=ax, cmap="Blues", xticks_rotation=45)
plt.title("Confusion Matrix – Variante B (CNN)")
plt.show()

***Ergebnisse – Ansatz B: Separate Klassifikation***

In [ ]:
def evaluate_model(model, model_type, category_name, data_variant, X_test, y_test):
    y_pred = model.predict(X_test)

    # Falls Wahrscheinlichkeiten, in Klassen umwandeln
    if y_pred.ndim > 1 and y_pred.shape[1] > 1:
        y_pred = np.argmax(y_pred, axis=1)

    # Falls y_test One-Hot, in Klassenlabels umwandeln
    if y_test.ndim > 1 and y_test.shape[1] > 1:
        y_test_labels = np.argmax(y_test, axis=1)
    else:
        y_test_labels = y_test

    acc = accuracy_score(y_test_labels, y_pred)
    report = classification_report(y_test_labels, y_pred, output_dict=True)

    result = {
        category_name: {
            'Model': model_type,
            'Category': category_name,
            'Data_Variant': data_variant,
            'Accuracy': round(acc, 4),
            'Precision': round(report['macro avg']['precision'], 4),
            'Recall': round(report['macro avg']['recall'], 4),
            'F1-Score': round(report['macro avg']['f1-score'], 4),
        }
    }
    return result[category_name]

def evaluate_all_models_combined(data_variants, model_types, version='v2'):
    results = []

    for model_type in model_types:
        for variant in data_variants:
            print(f"{model_type.upper()} | {variant} | Version: {version}")
            base_path = f"saved_models/{version}/{model_type}/"
            models_to_evaluate = []

            if model_type == 'ml':
                model_path = f"{base_path}best_model_{variant}.pkl"
                try:
                    model = joblib.load(model_path)
                    models_to_evaluate.append((f"ML - {variant}", model))
                except Exception as e:
                    print(f" ML-Modell nicht gefunden: {model_path}")

            elif model_type == 'ann':
                model_path = f"{base_path}model_{variant}.keras"
                try:
                    model = load_model(model_path)
                    models_to_evaluate.append((f"ANN - {variant}", model))
                except Exception as e:
                    print(f" ANN-Modell nicht gefunden: {model_path}")

            elif model_type == 'cnn':
                model_path_no_aug = f"{base_path}model_without_augmentation_{variant}.keras"
                model_path_aug = f"{base_path}model_with_augmentation_{variant}.keras"

                try:
                    model = load_model(model_path_no_aug)
                    models_to_evaluate.append((f"CNN (ohne Aug.) - {variant}", model))
                except Exception as e:
                    print(f"CNN ohne Augmentierung nicht gefunden: {model_path_no_aug}")

                try:
                    model = load_model(model_path_aug)
                    models_to_evaluate.append((f"CNN (mit Aug.) - {variant}", model))
                except Exception as e:
                    print(f"CNN mit Augmentierung nicht gefunden: {model_path_aug}")

            # Daten vorbereiten
            if variant == 'gray':
                X_train, X_test, _, y_test = prepare_combined_data(X_gray, y_full_gray)
            else:
                X_train, X_test, _, y_test = prepare_combined_data(X_wavelet, y_full_wavelet)

            if model_type == 'ann':
                scaler = StandardScaler()
                X_train = scaler.fit_transform(X_train)
                X_test = scaler.transform(X_test)

            if model_type == 'cnn':
                X_test = X_test.reshape(-1, global_image_size[0], global_image_size[1], 1) / 255.0

            for model_label, model in models_to_evaluate:
                result = evaluate_model(model, model_label, 'Combined', variant, X_test, y_test)
                results.append(result)

    return results

In [ ]:

data_variants = ['gray', 'wavelet']
model_types = ['ml', 'ann', 'cnn']

results_v2_combined = evaluate_all_models_combined(data_variants, model_types)

# Als DataFrame
df_results_v2_combined = pd.DataFrame(results_v2_combined)

# Optional gestylt
df_results_v2_combined.style.set_caption("Tabelle 4.2: Vergleich der Modelle – Variante B (kombinierte Klassifikation)") \
    .set_table_styles([{
        'selector': 'caption',
        'props': [('color', 'darkblue'), ('font-size', '15px')]
    }])

###  Beispielhafte Vorhersagen


***Ansatz A (Separate Klassifikation – Tomato, CNN)***
- **Original-Kategorie:** Tomato Late_blight  

In [ ]:

def predict_and_display(img_path, model_path, labels, global_image_size=(64, 64)):
    # Bild vorbereiten
    img_gray = Image.open(img_path).convert('L')
    img_gray_resized = img_gray.resize(global_image_size)
    img_array = np.array(img_gray_resized).reshape(1, global_image_size[0], global_image_size[1], 1) / 255.0

    # Modell laden und Vorhersage machen
    model = load_model(model_path)
    prediction = model.predict(img_array)
    predicted_class = np.argmax(prediction)

    # Originalbild laden und umwandeln
    original = cv2.imread(img_path)
    original = cv2.cvtColor(original, cv2.COLOR_BGR2RGB)

    # Anzeige nebeneinander (horizontal)
    fig, axs = plt.subplots(1, 2, figsize=(10, 4))

    axs[0].imshow(original)
    axs[0].set_title("Originalbild")
    axs[0].axis('off')

    axs[1].imshow(img_gray_resized, cmap='gray')
    axs[1].set_title(f"Vorverarbeitet\n(Predicted: {labels[predicted_class]})")
    axs[1].axis('off')

    plt.tight_layout()
    plt.show()


In [ ]:
labels_tomato = ['Bacterial_spot', 'Late_blight', 'healthy']
img_path = 'data/PlantVillage/Tomato___Late_blight/image (8).JPG'
model_path = 'saved_models/v1/cnn/tomato/model_without_augmentation_Tomato_gray.keras'

predict_and_display(img_path, model_path, labels_tomato)

***Ansatz B (Kombinierte Klassifikation )***
- **Original-Kategorie:** Corn  

In [ ]:
labels = [
    'Apple___Apple_scab',
    'Apple___Black_rot',
    'Apple___healthy',
    'Corn___Common_rust',
    'Corn___Northern_Leaf_Blight',
    'Corn___healthy',
    'Tomato___Bacterial_spot',
    'Tomato___Late_blight',
    'Tomato___healthy'
]
img_path = 'data/PlantVillage/Corn___Northern_Leaf_Blight/image (31).JPG'
img_path=  'data/PlantVillage/Apple___Black_rot/image (11).JPG'
model_path = 'saved_models/v2/cnn/model_without_augmentation_gray.keras'
predict_and_display(img_path, model_path, labels)